# Data models and context broker
This notebook shows how to create data models with the toolbox components and how to post and retrieve them from a context broker.

## Data Models

For each one of the defined toolbox [data models](../DataModels/README.md) there is an interface class that allow us to store its data and facilitates the communication with the context broker.

In [ ]:
import json
import pprint
import numpy as np
import requests
import pprint

from toolbox import Structures
from toolbox.Structures.Keypoints import COCOKeypoints

from toolbox import DataModels

# We can get the data model class by its type name using the data models catalog
data_models_dict = DataModels.DataModelsCatalog.data_models_catalog
pprint.pprint(data_models_dict)
print(data_models_dict["Face"])


### Create some data model

In [ ]:
face = DataModels.Face(
    id="Face:123",
    image="Image:123",
    boundingBox=Structures.BoundingBox(.1,.1,.2,.2),
    detectionConfidence=0.9,
    age=35,
    gender=Structures.Gender.FEMALE,
    genderConfidence=0.99,
    emotion=Structures.Emotion.HAPPINESS,
    emotionConfidence=0.99,
    features=[0.1,0.2,0.3],
    featuresAlgorithm="FaceNet",
    recognitionDomain="Celeb",
    recognized=True,
    recognizedDistance=0.77,
    recognizedPerson="Person:123"
)

instance_segmentation = DataModels.InstanceSegmentation(
    id="Instance:camera123",
    image="Image:123",
    mask=Structures.SegmentationMask(np.zeros((512,512))),
    boundingBox=Structures.BoundingBox(.1,.1,.2,.2),
    label="cat",
    labelId=0,
    confidence=0.99
)

keypoints = DataModels.PersonKeyPoints(
    id="KeyPoints:123",
    image="Image:123",
    boundingBox=Structures.BoundingBox(.1,.1,.2,.2),
    confidence=0.99,
    keypoints=COCOKeypoints.from_absolute_keypoints(np.full((17,3),0.5), 512, 512)
)

image = DataModels.Image(
    id="Image:123",
    width=512,
    height=512,
    path="/a/b/c.jpg",
    url="http://a.b/c.jpg",
    source="Camera:123",
    purpose="FaceRecognition"
)

We can pretty print the data models with the ``pretty()`` method

In [ ]:
print(face.pretty())
print(instance_segmentation.pretty())
print(keypoints.pretty())
print(image.pretty())

We can get and modify the data models attributes by its name

In [ ]:
face.age = 20
print(face.age)

## Posting and retrieving data models from the context broker

We can use the ``ContextProducer`` and ``ContextConsumer`` components to post and retrieve the defined data models from a context broker

In [ ]:
from toolbox.Context import ContextConsumer, ContextProducer

In [ ]:
config = {
    "context_broker": {
        "host": "192.168.0.69",
        "port": 1026
    }
}
context_producer = ContextProducer(config)
context_consumer = ContextConsumer(config)

Post one of the created data models

In [ ]:
dm = face

# If the id of the data model is not set, the ``ContextProducer`` will generate
# a random one before posting it to the broker. We can set the id to None
# to ensure that we don't use an existing one.
dm.id = None

# Post the data model to the context broker
dm_sent = context_producer.post_entity(dm)

# This returns a dict with the json that is sent to the context broker
print(type(dm_sent))
pprint.pprint(dm_sent)

If we want, we can also parse the data model to a json without posting it to the broke

In [ ]:
dm_json = context_producer.to_json(dm)
print(type(dm_json))
pprint.pprint(dm_json)

We can retrieve the data model we just posted by its id

In [ ]:
dm_retrieved = context_consumer.parse_entity(dm_sent["id"])
print(dm_retrieved.pretty())

In [ ]:
context_broker_host = f"{config['context_broker']['host']}:{config['context_broker']['port']}"

### NGSI-LD normalized

In [ ]:
r = requests.get(f"http://{context_broker_host}/ngsi-ld/v1/entities/{dm_sent['id']}")
print(r.status_code)
print(json.dumps(r.json(), indent=2))

### NGSI-LD key-values

In [ ]:
r = requests.get(f"http://{context_broker_host}/ngsi-ld/v1/entities/{dm_sent['id']}?options=keyValues")
print(r.status_code)
print(json.dumps(r.json(), indent=2))

### NGSIv2 normalized

In [ ]:
r = requests.get(f"http://{context_broker_host}/v2/entities/{dm_sent['id']}")
print(r.status_code)
print(json.dumps(r.json(), indent=2,))

### NGSIv2 key-values

In [ ]:
r = requests.get(f"http://{context_broker_host}/v2/entities/{dm_sent['id']}?options=keyValues")
print(r.status_code)
print(json.dumps(r.json(), indent=2,))